In [1]:
# 나눔고딕 폰트 설치 및 설정
!apt-get update -qq
!apt-get install fonts-nanum -qq
!fc-cache -fv
!rm ~/.cache/matplotlib -rf

import matplotlib.pyplot as plt

# 폰트 설정
import matplotlib.font_manager as fm

font_path = '/usr/share/fonts/truetype/nanum/NanumGothic.ttf'
fontprop = fm.FontProperties(fname=font_path, size=10)
plt.rcParams['font.family'] = 'NanumGothic'
plt.rcParams['axes.unicode_minus'] = False

W: Skipping acquire of configured file 'main/source/Sources' as repository 'https://r2u.stat.illinois.edu/ubuntu jammy InRelease' does not seem to provide it (sources.list entry misspelt?)
Selecting previously unselected package fonts-nanum.
(Reading database ... 126380 files and directories currently installed.)
Preparing to unpack .../fonts-nanum_20200506-1_all.deb ...
Unpacking fonts-nanum (20200506-1) ...
Setting up fonts-nanum (20200506-1) ...
Processing triggers for fontconfig (2.13.1-4.2ubuntu5) ...
/usr/share/fonts: caching, new cache contents: 0 fonts, 1 dirs
/usr/share/fonts/truetype: caching, new cache contents: 0 fonts, 3 dirs
/usr/share/fonts/truetype/humor-sans: caching, new cache contents: 1 fonts, 0 dirs
/usr/share/fonts/truetype/liberation: caching, new cache contents: 16 fonts, 0 dirs
/usr/share/fonts/truetype/nanum: caching, new cache contents: 12 fonts, 0 dirs
/usr/local/share/fonts: caching, new cache contents: 0 fonts, 0 dirs
/root/.local/share/fonts: skipping, no

In [2]:
# K-Pop 가사 데이터셋을 활용한 GRU 음악 분류기

# 필요한 라이브러리 설치
!pip install konlpy
!pip install tensorflow
!pip install scikit-learn
!pip install pandas numpy matplotlib seaborn
!pip install requests beautifulsoup4
!pip install openpyxl
!pip install gitpython

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, GRU, Dense, Dropout, Bidirectional, LSTM
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau, ModelCheckpoint
from konlpy.tag import Okt
import re
import warnings
import os
import json
import git
from collections import Counter
warnings.filterwarnings('ignore')

# 한글 폰트 설정 (Colab용)
plt.rcParams['font.family'] = 'DejaVu Sans'

class KPopLyricsDataLoader:
    """K-Pop 가사 데이터 로더 (EX3exp/Kpop-lyric-datasets 활용)"""

    def __init__(self, repo_path="Kpop-lyric-datasets"):
        self.repo_path = repo_path
        self.repo_url = "https://github.com/EX3exp/Kpop-lyric-datasets.git"

    def clone_repository(self):
        """GitHub 레포지토리 클론"""
        if not os.path.exists(self.repo_path):
            print("K-Pop 가사 데이터셋 다운로드 중...")
            try:
                # Git이 설치되어 있는지 확인
                import subprocess
                result = subprocess.run(['git', '--version'], capture_output=True, text=True)
                if result.returncode != 0:
                    raise Exception("Git이 설치되어 있지 않습니다.")

                # 레포지토리 클론
                git.Repo.clone_from(self.repo_url, self.repo_path)
                print("데이터셋 다운로드 완료!")
                return True

            except ImportError:
                print(" gitpython 라이브러리가 없습니다.")
                print("다음 명령어로 설치해주세요: !pip install gitpython")
                return False

            except Exception as e:
                print(f" 다운로드 실패: {e}")
                print("\n 수동 해결 방법:")
                print("1. Colab에서 다음 명령어 실행:")
                print(f"   !git clone {self.repo_url}")
                print("2. 또는 직접 GitHub에서 다운로드:")
                print(f"   {self.repo_url}")
                print("3. 압축 해제 후 'Kpop-lyric-datasets' 폴더명 확인")
                return False
        else:
            print(" 데이터셋이 이미 존재합니다.")
            return True

    def load_data_parser_method(self, start_year=2010, end_year=2023):
        """데이터 파서를 사용한 데이터 로드 (원본 방식)"""
        try:
            # utils 모듈 임포트 시도
            import sys
            sys.path.append(self.repo_path)
            from utils import data_parser

            print(f"{start_year}년부터 {end_year}년까지의 데이터 로드 중...")
            df = data_parser.get_df(start_year, end_year)
            print(f" 데이터 로드 완료: {len(df)}곡")
            return df

        except ImportError as e:
            print(f" utils 모듈 로드 실패: {e}")
            print("대체 방법으로 데이터를 로드합니다...")
            return self.load_data_manual_method(start_year, end_year)

    def load_data_manual_method(self, start_year=2010, end_year=2023):
        """수동으로 JSON 파일들을 파싱하여 데이터 로드"""
        print("수동 방식으로 JSON 파일들을 파싱 중...")

        data_list = []
        base_path = os.path.join(self.repo_path, "melon", "monthly-chart")

        if not os.path.exists(base_path):
            print(f" 경로를 찾을 수 없습니다: {base_path}")
            return self.create_sample_data()

        # 연도별 폴더 탐색
        for year_folder in os.listdir(base_path):
            try:
                year = int(year_folder.split('-')[-1])  # melon-2020 -> 2020
                if year < start_year or year > end_year:
                    continue

                year_path = os.path.join(base_path, year_folder)
                if not os.path.isdir(year_path):
                    continue

                print(f" {year}년 데이터 처리 중...")

                # 월별 폴더 탐색
                for month_folder in os.listdir(year_path):
                    month_path = os.path.join(year_path, month_folder)
                    if not os.path.isdir(month_path):
                        continue

                    # JSON 파일들 처리
                    for json_file in os.listdir(month_path):
                        if not json_file.endswith('.json'):
                            continue

                        file_path = os.path.join(month_path, json_file)
                        try:
                            with open(file_path, 'r', encoding='utf-8') as f:
                                song_data = json.load(f)

                                # 가사 텍스트 추출
                                lyrics_lines = song_data.get('lyrics', {}).get('lines', [])
                                lyrics_text = ' '.join([line for line in lyrics_lines if line.strip()])

                                # 빈 가사 제외
                                if len(lyrics_text.strip()) < 10:
                                    continue

                                # 데이터 추출
                                data_list.append({
                                    'title': song_data.get('song_name', ''),
                                    'artist': song_data.get('artist', ''),
                                    'lyrics': lyrics_text,
                                    'genre': song_data.get('genre', ''),
                                    'album': song_data.get('album', ''),
                                    'release_date': song_data.get('release_date', ''),
                                    'year': song_data.get('info', [{}])[0].get('year', year),
                                    'month': song_data.get('info', [{}])[0].get('month', 1),
                                    'rank': song_data.get('info', [{}])[0].get('rank', 0),
                                    'lyric_writer': song_data.get('lyric_writer', ''),
                                    'composer': song_data.get('composer', '')
                                })

                        except (json.JSONDecodeError, KeyError, Exception) as e:
                            print(f"  파일 처리 오류 ({json_file}): {e}")
                            continue

            except (ValueError, Exception) as e:
                print(f"  연도 폴더 처리 오류 ({year_folder}): {e}")
                continue

        if not data_list:
            print(" 데이터를 로드할 수 없습니다. 샘플 데이터를 생성합니다.")
            return self.create_sample_data()

        df = pd.DataFrame(data_list)
        print(f" 수동 로드 완료: {len(df)}곡")
        return df

    def load_kpop_dataset(self, start_year=2010, end_year=2023):
        """K-Pop 데이터셋 로드 (메인 함수)"""
        # 1. 레포지토리 클론
        if not self.clone_repository():
            return self.create_sample_data()

        # 2. 데이터 파서 방식 시도
        try:
            df = self.load_data_parser_method(start_year, end_year)
            if df is not None and len(df) > 0:
                return self.process_dataframe(df)
        except Exception as e:
            print(f"데이터 파서 방식 실패: {e}")

        # 3. 수동 방식으로 대체
        df = self.load_data_manual_method(start_year, end_year)
        return self.process_dataframe(df)

    def process_dataframe(self, df):
        """데이터프레임 후처리"""
        print(" 데이터 후처리 중...")

        # 중복 제거
        initial_count = len(df)
        df = df.drop_duplicates(subset=['title', 'artist'], keep='first')
        print(f"중복 제거: {initial_count} → {len(df)}곡")

        # 빈 값 처리
        df = df.dropna(subset=['lyrics', 'genre'])
        df = df[df['lyrics'].str.len() > 20]  # 너무 짧은 가사 제외

        # 장르 정리
        df['genre'] = df['genre'].str.strip()
        genre_counts = df['genre'].value_counts()

        # 최소 곡 수 이상인 장르만 유지 (분류 성능을 위해)
        min_songs = 50
        valid_genres = genre_counts[genre_counts >= min_songs].index
        df = df[df['genre'].isin(valid_genres)]

        print(f"최종 데이터: {len(df)}곡")
        print(f"장르별 분포:\n{df['genre'].value_counts()}")

        return df

    def create_sample_data(self):
        """샘플 데이터 생성 (백업용)"""
        print("  실제 데이터를 로드할 수 없어 샘플 데이터를 생성합니다.")

        sample_data = {
            'title': ['샘플곡1', '샘플곡2', '샘플곡3', '샘플곡4', '샘플곡5'] * 10,
            'artist': ['아티스트A', '아티스트B', '아티스트C', '아티스트D', '아티스트E'] * 10,
            'lyrics': [
                '사랑하는 마음이 전해지길 바라며 오늘도 노래해',
                '신나는 음악에 맞춰 모두 함께 춤춰봐 즐거운 하루',
                '힘든 세상 속에서도 꿈을 포기하지 않고 앞으로 나아가',
                '조용한 밤 혼자 앉아 그리운 사람을 생각해',
                '랩으로 전하는 진실한 이야기 들어봐'
            ] * 10,
            'genre': ['발라드', '댄스', '힙합', '인디', '랩'] * 10,
            'year': [2020, 2021, 2022, 2023, 2024] * 10
        }

        return pd.DataFrame(sample_data)

class KoreanLyricsClassifier:
    def __init__(self, max_features=20000, max_length=400, embedding_dim=150):
        self.max_features = max_features
        self.max_length = max_length
        self.embedding_dim = embedding_dim
        self.tokenizer = Tokenizer(num_words=max_features, oov_token="<OOV>")
        self.okt = Okt()
        self.label_encoder = LabelEncoder()
        self.model = None
        self.history = None

    def preprocess_text(self, text):
        """한글 텍스트 전처리 (개선된 버전)"""
        if pd.isna(text) or text == "":
            return ""

        # 특수문자 제거 (한글, 영어, 숫자, 공백만 유지)
        text = re.sub(r'[^가-힣a-zA-Z0-9\s]', '', str(text))

        # 길이 제한 (너무 긴 텍스트 방지)
        if len(text) > 5000:
            text = text[:5000]

        try:
            # 형태소 분석 및 품사 태깅
            morphs = self.okt.pos(text, stem=True)

            # 의미있는 품사만 선별 (명사, 동사, 형용사, 부사)
            meaningful_words = []
            for word, pos in morphs:
                if (pos in ['Noun', 'Verb', 'Adjective', 'Adverb'] and
                    len(word) > 1 and
                    not word.isdigit()):
                    meaningful_words.append(word)

            return ' '.join(meaningful_words)

        except Exception as e:
            print(f"형태소 분석 오류: {e}")
            # 형태소 분석 실패시 간단한 전처리만 수행
            words = text.split()
            return ' '.join([word for word in words if len(word) > 1])

    def prepare_data(self, df):
        """데이터 전처리 및 준비 (K-Pop 데이터 최적화)"""
        print(" 데이터 전처리 시작...")

        # 필수 컬럼 확인
        if 'lyrics' not in df.columns or 'genre' not in df.columns:
            print(" 필수 컬럼 (lyrics, genre)이 없습니다.")
            print(f"현재 컬럼: {df.columns.tolist()}")
            return None, None

        # 데이터 정제
        print(" 데이터 정제 중...")
        df = df.dropna(subset=['lyrics', 'genre'])
        df = df[df['lyrics'].str.len() > 20]  # 최소 길이 확보

        print(f"정제 후 데이터 크기: {len(df)}")

        # 장르별 분포 확인 및 균형 맞추기
        genre_counts = df['genre'].value_counts()
        print(f"장르별 분포:\n{genre_counts}")

        # 각 장르별로 최소 30개 이상의 곡이 있는지 확인
        min_songs_per_genre = 30
        valid_genres = genre_counts[genre_counts >= min_songs_per_genre].index
        df = df[df['genre'].isin(valid_genres)]

        if len(df) < 100:
            print(" 충분한 데이터가 없습니다. 최소 100곡 이상 필요합니다.")
            return None, None

        print(f"최종 데이터 크기: {len(df)}")
        print(f"사용할 장르: {list(valid_genres)}")

        # 텍스트 전처리
        print(" 텍스트 전처리 중...")
        df['processed_lyrics'] = df['lyrics'].apply(self.preprocess_text)

        # 전처리 후 빈 문자열 제거
        df = df[df['processed_lyrics'].str.len() > 5]

        print(f"전처리 후 데이터 크기: {len(df)}")

        # 토크나이저 학습
        print(" 토크나이저 학습 중...")
        self.tokenizer.fit_on_texts(df['processed_lyrics'])

        # 어휘 크기 정보
        word_index = self.tokenizer.word_index
        print(f"전체 어휘 크기: {len(word_index)}")
        print(f"사용할 어휘 크기: {min(len(word_index), self.max_features)}")

        # 텍스트를 시퀀스로 변환
        print(" 시퀀스 변환 중...")
        sequences = self.tokenizer.texts_to_sequences(df['processed_lyrics'])
        X = pad_sequences(sequences, maxlen=self.max_length, padding='post', truncating='post')

        # 레이블 인코딩
        y = self.label_encoder.fit_transform(df['genre'])

        # 시퀀스 길이 분석
        seq_lengths = [len(seq) for seq in sequences]
        print(f"시퀀스 길이 - 평균: {np.mean(seq_lengths):.1f}, "
              f"중간값: {np.median(seq_lengths):.1f}, "
              f"최대: {np.max(seq_lengths)}")

        # 상위 빈도 단어 출력
        print("\n상위 빈도 단어:")
        word_freq = Counter()
        for seq in sequences:
            word_freq.update(seq)

        top_words = word_freq.most_common(10)
        word_to_index = {v: k for k, v in word_index.items()}
        for word_id, freq in top_words:
            if word_id in word_to_index:
                print(f"  {word_to_index[word_id]}: {freq}")

        return X, y

    def build_advanced_model(self, num_classes):
        """K-Pop 데이터에 최적화된 고급 GRU 모델"""
        model = Sequential([
            # Embedding Layer
            Embedding(
                input_dim=self.max_features,
                output_dim=self.embedding_dim,
                input_length=self.max_length,
                mask_zero=True
            ),

            # Multi-layer Bidirectional GRU
            Bidirectional(GRU(256, return_sequences=True, dropout=0.3, recurrent_dropout=0.2)),
            Bidirectional(GRU(128, return_sequences=True, dropout=0.3, recurrent_dropout=0.2)),
            Bidirectional(GRU(64, dropout=0.3, recurrent_dropout=0.2)),

            # Dense Layers with Regularization
            Dense(256, activation='relu'),
            Dropout(0.5),
            Dense(128, activation='relu'),
            Dropout(0.4),
            Dense(64, activation='relu'),
            Dropout(0.3),

            # Output Layer
            Dense(num_classes, activation='softmax')
        ])

        # 모델 빌드 (input_shape 명시적 지정)
        model.build(input_shape=(None, self.max_length))

        # 최적화된 컴파일 설정
        optimizer = tf.keras.optimizers.Adam(
            learning_rate=0.001,
            clipnorm=1.0  # 그래디언트 클리핑
        )

        # 클래스 수에 따라 메트릭 선택
        metrics = ['accuracy']
        if num_classes > 3:  # 클래스가 3개 이상일 때만 top-k 메트릭 추가
            metrics.append(tf.keras.metrics.SparseTopKCategoricalAccuracy(k=min(3, num_classes-1), name='top_k_accuracy'))

        model.compile(
            optimizer=optimizer,
            loss='sparse_categorical_crossentropy',
            metrics=metrics
        )
        self.model = model
        return model


    def build_simple_model(self, num_classes):
        """간단한 GRU 모델 (백업용)"""
        try:
            print(" 간단한 모델로 대체합니다...")

            model = Sequential([
                Embedding(
                    input_dim=self.max_features,
                    output_dim=64,  # 더 작은 임베딩 차원
                    input_length=self.max_length
                ),
                GRU(128, dropout=0.3, recurrent_dropout=0.2),
                Dense(64, activation='relu'),
                Dropout(0.5),
                Dense(num_classes, activation='softmax')
            ])

            # 모델 빌드
            model.build(input_shape=(None, self.max_length))

            # 간단한 컴파일
            model.compile(
                optimizer='adam',
                loss='sparse_categorical_crossentropy',
                metrics=['accuracy']
            )

            self.model = model
            print(" 간단한 GRU 모델 생성 성공")
            return model

        except Exception as e:
            print(f" 간단한 모델 생성도 실패: {e}")
            self.model = None
            raise e

    def build_basic_model(self, num_classes):
        """기본 모델 (백업용) - build_advanced_model과 build_simple_model 사이에 추가"""
        try:
            print(" 기본 모델로 대체합니다...")

            model = Sequential([
                Embedding(
                    input_dim=self.max_features,
                    output_dim=100, # 중간 임베딩 차원
                    input_length=self.max_length
                ),
                GRU(192, dropout=0.3, recurrent_dropout=0.2), # 중간 GRU 유닛
                Dense(100, activation='relu'),
                Dropout(0.5),
                Dense(num_classes, activation='softmax')
            ])

            # 모델 빌드
            model.build(input_shape=(None, self.max_length))

            # 컴파일
            model.compile(
                optimizer='adam',
                loss='sparse_categorical_crossentropy',
                metrics=['accuracy']
            )

            self.model = model
            print(" 기본 GRU 모델 생성 성공")
            return model

        except Exception as e:
            print(f" 기본 모델 생성 실패: {e}")
            self.model = None
            raise e


    def build_ultra_simple_model(self, num_classes):
        """가장 간단한 모델 (최종 백업)"""
        try:
            print(" 가장 간단한 모델을 생성합니다...")

            # 최소한의 설정
            vocab_size = min(1000, self.max_features)
            seq_length = min(50, self.max_length)

            model = Sequential()
            model.add(Embedding(vocab_size, 16, input_length=seq_length))
            model.add(GRU(32))
            model.add(Dense(num_classes, activation='softmax'))

            # 수동으로 빌드
            model.build(input_shape=(None, seq_length))

            # 컴파일
            model.compile(
                optimizer='adam',
                loss='sparse_categorical_crossentropy',
                metrics=['accuracy']
            )

            self.model = model
            print(" 초간단 모델 생성 성공")
            return model

        except Exception as e:
            print(f" 초간단 모델도 실패: {e}")
            self.model = None
            return None

    def train_with_kpop_data(self, X, y, validation_split=0.2, epochs=100, batch_size=32):
        """K-Pop 데이터에 최적화된 훈련"""
        print(" K-Pop 가사 분류 모델 훈련 시작...")

        # 모델이 제대로 생성되었는지 확인
        if self.model is None:
            print(" 모델이 생성되지 않았습니다. 훈련을 중단합니다.")
            raise ValueError("모델이 None입니다. 먼저 모델을 생성해주세요.")

        # 고급 콜백 설정
        callbacks = [
            EarlyStopping(
                monitor='val_loss',
                patience=20,
                restore_best_weights=True,
                verbose=1
            ),
            ReduceLROnPlateau(
                monitor='val_loss',
                factor=0.3,
                patience=10,
                min_lr=0.00001,
                verbose=1
            ),
            ModelCheckpoint(
                'best_kpop_classifier.h5',
                monitor='val_accuracy',
                save_best_only=True,
                verbose=1
            )
        ]

        # 클래스 가중치 계산 (불균형 데이터 대응)
        try:
            from sklearn.utils.class_weight import compute_class_weight

            class_weights = compute_class_weight(
                'balanced',
                classes=np.unique(y),
                y=y
            )
            class_weight_dict = dict(enumerate(class_weights))
            print(f"클래스 가중치 적용: {len(class_weight_dict)}개 클래스")

        except Exception as e:
            print(f"  클래스 가중치 계산 실패: {e}")
            print("균등 가중치로 훈련을 진행합니다.")
            class_weight_dict = None

        # 모델 훈련
        try:
            history = self.model.fit(
                X, y,
                validation_split=validation_split,
                epochs=epochs,
                batch_size=batch_size,
                callbacks=callbacks,
                class_weight=class_weight_dict,
                verbose=1
            )

            self.history = history
            print(" 모델 훈련 완료!")
            return history

        except Exception as e:
            print(f" 훈련 중 오류 발생: {e}")
            print("배치 크기를 줄이거나 에포크 수를 줄여서 다시 시도해보세요.")
            raise e

    def predict_song_genre(self, lyrics):
        """가사로 장르 예측"""
        if self.model is None:
            return "모델 없음", 0.0, [("모델 없음", 0.0)]

        processed_lyrics = self.preprocess_text(lyrics)
        if not processed_lyrics:
            return "알 수 없음", 0.0, [("알 수 없음", 0.0)]

        try:
            sequence = self.tokenizer.texts_to_sequences([processed_lyrics])
            padded_sequence = pad_sequences(sequence, maxlen=self.max_length, padding='post', truncating='post')

            prediction = self.model.predict(padded_sequence, verbose=0)
            predicted_class = np.argmax(prediction, axis=1)[0]
            confidence = np.max(prediction)

            genre = self.label_encoder.inverse_transform([predicted_class])[0]

            # Top 3 예측 결과도 반환
            top_3_indices = np.argsort(prediction[0])[-3:][::-1]
            top_3_genres = [(self.label_encoder.inverse_transform([idx])[0], prediction[0][idx])
                           for idx in top_3_indices]

            return genre, confidence, top_3_genres

        except Exception as e:
            print(f"예측 오류: {e}")
            return "예측 실패", 0.0, [("예측 실패", 0.0)]

    def analyze_model_performance(self, X_test, y_test):
        """모델 성능 상세 분석"""
        print("\n 모델 성능 분석 시작...")

        # 기본 평가
        evaluation_results = self.model.evaluate(X_test, y_test, verbose=0)

        # 결과 파싱
        if isinstance(evaluation_results, list):
            test_loss = evaluation_results[0]
            test_accuracy = evaluation_results[1]
            # top-k 정확도가 있는 경우
            test_top_k = evaluation_results[2] if len(evaluation_results) > 2 else None
        else:
            test_loss = evaluation_results
            test_accuracy = None
            test_top_k = None


        print(f"테스트 손실: {test_loss:.4f}")
        if test_accuracy:
            print(f"테스트 정확도: {test_accuracy:.4f}")
        if test_top_k:
            print(f"Top-K 정확도: {test_top_k:.4f}")

        # 예측 수행
        y_pred = self.model.predict(X_test, verbose=0)
        y_pred_classes = np.argmax(y_pred, axis=1)

        # 장르별 상세 분석
        target_names = self.label_encoder.classes_
        print("\n 장르별 분류 리포트:")
        print(classification_report(y_test, y_pred_classes, target_names=target_names))

        # 혼동 행렬 시각화
        plt.figure(figsize=(12, 10))
        cm = confusion_matrix(y_test, y_pred_classes)
        sns.heatmap(cm, annot=True, fmt='d', cmap='Blues',
                    xticklabels=target_names, yticklabels=target_names)
        plt.title('K-Pop 장르 분류 혼동 행렬')
        plt.xlabel('예측 장르')
        plt.ylabel('실제 장르')
        plt.xticks(rotation=45)
        plt.yticks(rotation=0)
        plt.tight_layout()
        plt.show()

        return test_accuracy if test_accuracy else test_loss, y_pred_classes

    def plot_training_history_advanced(self):
        """고급 훈련 과정 시각화"""
        if self.history is None:
            print("훈련 기록이 없습니다.")
            return

        # 사용 가능한 메트릭 확인
        available_metrics = list(self.history.history.keys())
        has_top_k = any('top_k' in metric for metric in available_metrics)

        # 서브플롯 개수 결정
        if has_top_k:
            fig, axes = plt.subplots(2, 2, figsize=(15, 10))
        else:
            fig, axes = plt.subplots(1, 2, figsize=(15, 5))
            # 1x2 배치인 경우 axes를 2D로 변환
            axes = axes.reshape(1, -1)

        # 손실 그래프
        axes[0, 0].plot(self.history.history['loss'], label='Training Loss', linewidth=2)
        axes[0, 0].plot(self.history.history['val_loss'], label='Validation Loss', linewidth=2)
        axes[0, 0].set_title('Model Loss')
        axes[0, 0].set_xlabel('Epoch')
        axes[0, 0].set_ylabel('Loss')
        axes[0, 0].legend()
        axes[0, 0].grid(True, alpha=0.3)

        # 정확도 그래프
        axes[0, 1].plot(self.history.history['accuracy'], label='Training Accuracy', linewidth=2)
        axes[0, 1].plot(self.history.history['val_accuracy'], label='Validation Accuracy', linewidth=2)
        axes[0, 1].set_title('Model Accuracy')
        axes[0, 1].set_xlabel('Epoch')
        axes[0, 1].set_ylabel('Accuracy')
        axes[0, 1].legend()
        axes[0, 1].grid(True, alpha=0.3)

        # Top-K 정확도 (있는 경우)
        if has_top_k:
            top_k_metric = [m for m in available_metrics if 'top_k' in m and 'val_' not in m][0]
            val_top_k_metric = f'val_{top_k_metric}'

            if top_k_metric in self.history.history and val_top_k_metric in self.history.history:
                axes[1, 0].plot(self.history.history[top_k_metric],
                               label='Training Top-K Acc', linewidth=2)
                axes[1, 0].plot(self.history.history[val_top_k_metric],
                               label='Validation Top-K Acc', linewidth=2)
                axes[1, 0].set_title('Top-K Accuracy')
                axes[1, 0].set_xlabel('Epoch')
                axes[1, 0].set_ylabel('Top-K Accuracy')
                axes[1, 0].legend()
                axes[1, 0].grid(True, alpha=0.3)

            # 성능 요약
            axes[1, 1].text(0.5, 0.5, 'K-Pop 가사 분류기\n성능 요약',
                            horizontalalignment='center', verticalalignment='center',
                            transform=axes[1, 1].transAxes, fontsize=14)
            axes[1, 1].axis('off')

        plt.suptitle('K-Pop 가사 분류 모델 훈련 결과', fontsize=16)
        plt.tight_layout()
        plt.show()

        # 최고 성능 출력
        best_val_acc = max(self.history.history['val_accuracy'])
        best_epoch = self.history.history['val_accuracy'].index(best_val_acc) + 1
        print(f" 최고 검증 정확도: {best_val_acc:.4f} (에포크 {best_epoch})")

        # Top-K 정확도 최고치 출력 (있는 경우)
        if has_top_k:
            top_k_metric = [m for m in available_metrics if 'val_' in m and 'top_k' in m]
            if top_k_metric:
                best_val_top_k = max(self.history.history[top_k_metric[0]])
                best_top_k_epoch = self.history.history[top_k_metric[0]].index(best_val_top_k) + 1
                print(f" 최고 Top-K 정확도: {best_val_top_k:.4f} (에포크 {best_top_k_epoch})")

def main():
    print(" K-Pop 가사 GRU 분류기 ")
    print("=" * 50)

    # 1. K-Pop 데이터셋 로드
    print(" K-Pop 가사 데이터셋 로드...")
    data_loader = KPopLyricsDataLoader()

    # 2010년부터 2023년까지의 데이터 로드
    df = data_loader.load_kpop_dataset(start_year=2010, end_year=2023)

    if df is None or len(df) == 0:
        print(" 데이터를 로드할 수 없습니다.")
        return None

    print(f"\n 로드된 데이터 정보:")
    print(f"  - 총 곡 수: {len(df):,}곡")
    print(f"  - 컬럼: {list(df.columns)}")
    print(f"  - 기간: {df['year'].min()}년 ~ {df['year'].max()}년")

    if 'genre' in df.columns:
        print(f"\n 장르별 분포:")
        genre_dist = df['genre'].value_counts()
        for genre, count in genre_dist.head(10).items():
            print(f"  - {genre}: {count:,}곡")

    # 2. 분류기 초기화 및 데이터 전처리
    print(f"\n 분류기 초기화...")
    classifier = KoreanLyricsClassifier(
        max_features=25000,  # K-Pop 데이터에 맞게 증가
        max_length=500,      # 가사 길이에 맞게 조정
        embedding_dim=200    # 임베딩 차원 증가
    )

    # 3. 데이터 전처리
    print(f"\n  데이터 전처리...")
    X, y = classifier.prepare_data(df)

    if X is None or y is None:
        print(" 데이터 전처리 실패")
        return None

    print(f"전처리 완료:")
    print(f"  - 입력 데이터 형태: {X.shape}")
    print(f"  - 클래스 수: {len(np.unique(y))}")
    print(f"  - 클래스: {list(classifier.label_encoder.classes_)}")

    # 4. 훈련/테스트 데이터 분할
    X_train, X_test, y_train, y_test = train_test_split(
        X, y, test_size=0.2, random_state=42, stratify=y
    )

    print(f"\n 데이터 분할:")
    print(f"  - 훈련 데이터: {X_train.shape[0]:,}곡")
    print(f"  - 테스트 데이터: {X_test.shape[0]:,}곡")

    # 5. 모델 구축 (4단계 백업 시스템)
    print(f"\n  GRU 모델 구축...")
    num_classes = len(np.unique(y))
    model = None

    # 1단계: 고급 모델 시도
    try:
        print("  고급 GRU 모델 시도...")
        model = classifier.build_advanced_model(num_classes)

    except Exception as e:
        print(f"  고급 모델 구축 실패: {e}")

        # 2단계: 간단한 모델 시도
        try:
            print("  간단한 GRU 모델 시도...")
            model = classifier.build_simple_model(num_classes)

        except Exception as e2:
            print(f"  간단한 모델 구축 실패: {e2}")

            # 3단계: 기본 모델 시도
            try:
                print("  기본 모델 시도...")
                model = classifier.build_basic_model(num_classes)

            except Exception as e3:
                print(f"  기본 모델 구축 실패: {e3}")

                # 4단계: 초간단 모델 시도
                try:
                    print("  초간단 모델 시도...")
                    model = classifier.build_ultra_simple_model(num_classes)

                except Exception as e4:
                    print(f" 모든 모델 구축 실패: {e4}")
                    print("\n 모든 모델 생성이 실패했습니다.")
                    print("환경 문제일 가능성이 높습니다. 다음을 시도해보세요:")
                    print("1. 런타임 재시작")
                    print("2. TensorFlow 재설치: !pip install --upgrade tensorflow")
                    print("3. 가상환경 재설정")

                    # 실패해도 데이터는 반환
                    return None, df

    # 모델 생성 성공 확인
    if model is None or classifier.model is None:
        print(" 모델 생성에 완전히 실패했습니다.")
        print("데이터만 반환합니다.")
        return None, df

    # 모델 정보 출력
    try:
        param_count = model.count_params()
        print(f"\n 모델 구축 성공!")
        print(f"모델 정보:")
        print(f"  - 파라미터 수: {param_count:,}")
        print(f"  - 입력 크기: {classifier.max_length}")
        print(f"  - 어휘 크기: {classifier.max_features}")
        print(f"  - 클래스 수: {num_classes}")
        print(f"  - 클래스: {list(classifier.label_encoder.classes_)}")

        # 모델 레이어 정보만 간단히 출력
        print(f"\n 모델 레이어:")
        for i, layer in enumerate(model.layers):
            print(f"  {i+1}. {layer.__class__.__name__}")

    except Exception as e:
        print(f"  모델 정보 출력 오류: {e}")
        print("모델은 정상적으로 생성되었지만 정보 출력에 문제가 있습니다.")
        print("훈련을 계속 진행합니다...")

    # 6. 모델 훈련
    if classifier.model is None:
        print(" 모델이 생성되지 않아 훈련을 건너뜁니다.")
        print("데이터 분석 결과만 제공합니다.")
        return None, df

    print(f"\n 모델 훈련 시작...")

    # 데이터 크기에 따른 하이퍼파라미터 조정
    data_size = len(X_train)
    if data_size < 1000:
        epochs, batch_size = 10, 4  # 매우 보수적인 설정
        print(f" 소규모 데이터 설정: {epochs} 에포크, 배치 크기 {batch_size}")
    elif data_size < 5000:
        epochs, batch_size = 20, 8
        print(f" 중간 규모 데이터 설정: {epochs} 에포크, 배치 크기 {batch_size}")
    else:
        epochs, batch_size = 30, 16  # 더 작은 배치로 안전하게
        print(f" 대규모 데이터 설정: {epochs} 에포크, 배치 크기 {batch_size}")

    history = None

    try:
        # 첫 번째 시도
        print(" 첫 번째 훈련 시도...")
        history = classifier.train_with_kpop_data(
            X_train, y_train,
            validation_split=0.15,
            epochs=epochs,
            batch_size=batch_size
        )

    except Exception as e:
        print(f"  첫 번째 훈련 시도 실패: {e}")
        print("더 보수적인 설정으로 재시도합니다...")

        try:
            # 두 번째 시도 (더 작은 배치, 적은 에포크)
            print(" 두 번째 훈련 시도...")
            history = classifier.train_with_kpop_data(
                X_train, y_train,
                validation_split=0.1,
                epochs=max(5, epochs//2),
                batch_size=max(2, batch_size//2)
            )

        except Exception as e2:
            print(f" 두 번째 훈련 시도도 실패: {e2}")
            print("훈련 없이 모델만 반환합니다.")
            return classifier, df

    if history is None:
        print(" 훈련에 완전히 실패했습니다.")
        print("훈련되지 않은 모델을 반환합니다.")
        return classifier, df

    # 7. 훈련 과정 시각화
    if history is not None:
        print(f"\n 훈련 과정 시각화...")
        try:
            classifier.plot_training_history_advanced()
        except Exception as e:
            print(f"  시각화 오류: {e}")
    else:
        print("  훈련 기록이 없어 시각화를 건너뜁니다.")

    # 8. 모델 평가
    if classifier.model is not None and history is not None:
        print(f"\n 모델 평가...")
        try:
            test_accuracy, y_pred = classifier.analyze_model_performance(X_test, y_test)
        except Exception as e:
            print(f"  모델 평가 오류: {e}")
            test_accuracy = 0.0
    else:
        print("  모델이나 훈련 기록이 없어 평가를 건너뜁니다.")
        test_accuracy = 0.0

    # 9. 실제 예측 테스트
    if classifier.model is not None:
        print(f"\n 가사 장르 예측 테스트:")

        test_cases = [
            "사랑하는 사람아 내 마음을 받아줘 영원히 함께하자",
            "모두 일어나 손을 들어 신나게 춤을 춰봐 파티타임",
            "힘든 세상 속에서 꿈을 잃지 말고 끝까지 버텨내자",
            "조용한 밤 기타 소리에 맞춰 부르는 노래",
            "마이크 잡고 무대 위에서 랩으로 외치는 진실"
        ]

        for i, lyrics in enumerate(test_cases, 1):
            try:
                genre, confidence, top_3 = classifier.predict_song_genre(lyrics)
                print(f"\n{i}. 테스트 가사: '{lyrics[:30]}...'")
                print(f"    예측 장르: {genre}")
                print(f"    신뢰도: {confidence:.3f}")
                print(f"    Top 3: {[(g, f'{c:.3f}') for g, c in top_3]}")
            except Exception as e:
                print(f"{i}. 예측 오류: {e}")
    else:
        print("  모델이 없어 예측 테스트를 건너뜁니다.")

    # 10. 모델 저장
    if classifier.model is not None:
        try:
            classifier.model.save('kpop_lyrics_classifier_advanced.h5')
            print(f"\n 모델 저장 완료: 'kpop_lyrics_classifier_advanced.h5'")
        except Exception as e:
            print(f"  모델 저장 오류: {e}")
    else:
        print("  저장할 모델이 없습니다.")

    # 11. 최종 결과
    if classifier.model is not None and history is not None:
        print(f"\n K-Pop 가사 분류기 학습 완료!")
        print(f"최종 테스트 정확도: {test_accuracy:.4f}")
        return classifier, df
    else:
        print(f"\n K-Pop 가사 분류기 학습 실패")
        print("모델이나 훈련이 완료되지 않았습니다.")
        return None, df

# 추가 유틸리티 함수들
def analyze_kpop_trends(df):
    """K-Pop 트렌드 분석"""
    if 'year' not in df.columns:
        return

    print(" K-Pop 장르 트렌드 분석")

    # 연도별 장르 분포
    plt.figure(figsize=(15, 8))

    pivot_data = df.groupby(['year', 'genre']).size().unstack(fill_value=0)
    pivot_data.plot(kind='area', stacked=True, alpha=0.7)

    plt.title('연도별 K-Pop 장르 분포 변화')
    plt.xlabel('연도')
    plt.ylabel('곡 수')
    plt.legend(bbox_to_anchor=(1.05, 1), loc='upper left')
    plt.tight_layout()
    plt.show()

def export_predictions(classifier, df, output_file='kpop_predictions.csv'):
    """예측 결과를 CSV로 내보내기"""
    predictions = []

    for idx, row in df.iterrows():
        genre, confidence, top_3 = classifier.predict_song_genre(row['lyrics'])
        predictions.append({
            'title': row['title'],
            'artist': row['artist'],
            'actual_genre': row['genre'],
            'predicted_genre': genre,
            'confidence': confidence,
            'top_3_predictions': str(top_3)
        })

    pred_df = pd.DataFrame(predictions)
    pred_df.to_csv(output_file, index=False, encoding='utf-8-sig')
    print(f" 예측 결과 저장: {output_file}")

# 간단한 테스트 함수 추가
def quick_test():
    """빠른 테스트용 함수 (최소 설정)"""
    print(" 빠른 테스트 모드 시작...")

    try:
        # 최소 설정으로 분류기 생성
        classifier = KoreanLyricsClassifier(
            max_features=1000,
            max_length=50,
            embedding_dim=32
        )

        # 샘플 데이터 생성
        data_loader = KPopLyricsDataLoader()
        df = data_loader.create_sample_data()

        print(f"샘플 데이터: {len(df)}곡")

        # 데이터 전처리
        X, y = classifier.prepare_data(df)
        if X is None:
            print(" 데이터 전처리 실패")
            return None, df

        # 초간단 모델 생성
        num_classes = len(np.unique(y))
        model = classifier.build_ultra_simple_model(num_classes)

        if model is None:
            print(" 모델 생성 실패")
            return None, df

        print(" 빠른 테스트 성공!")
        print("실제 데이터로 main() 함수를 실행해보세요.")

        return classifier, df

    except Exception as e:
        print(f" 빠른 테스트 실패: {e}")
        return None, None

def safe_main():
    """안전한 main 함수 (오류 발생시 None, None 반환)"""
    try:
        return main()
    except Exception as e:
        print(f" main() 함수 실행 실패: {e}")
        print("\n 해결 방법:")
        print("1. quick_test() 함수로 먼저 테스트")
        print("2. 런타임 재시작 후 재실행")
        print("3. 라이브러리 재설치")
        return None, None

# 실행 코드
if __name__ == "__main__":
    # TensorFlow GPU 설정 (선택사항)
    try:
        gpus = tf.config.experimental.list_physical_devices('GPU')
        if gpus:
            tf.config.experimental.set_memory_growth(gpus[0], True)
            print(" GPU 사용 가능")
        else:
            print(" CPU 모드로 실행")
    except Exception as e:
        print(f"  GPU 설정 오류: {e}")
        print(" CPU 모드로 실행")


    try:
        # 안전한 메인 함수 실행
        result = safe_main()

        if result is not None and result[0] is not None and result[1] is not None:
            classifier, dataset = result
            print("\n" + "="*50)
            print(" 추가 분석 실행...")

            # K-Pop 트렌드 분석
            try:
                if 'year' in dataset.columns:
                    analyze_kpop_trends(dataset)
                else:
                    print("  연도 정보가 없어 트렌드 분석을 건너뜁니다.")
            except Exception as e:
                print(f"  트렌드 분석 오류: {e}")

            print("\n 실행 완료!")

        else:
            print("\n 실행 실패")
            print(" 빠른 테스트를 시도해보세요:")
            print("quick_test()")

    except KeyboardInterrupt:
        print("\n  사용자에 의해 중단되었습니다.")

    except ImportError as e:
        print(f" 라이브러리 import 오류: {e}")
        print("\n 해결 방법:")
        print("1. 필수 라이브러리 설치:")
        print("   !pip install konlpy tensorflow scikit-learn pandas numpy matplotlib seaborn")
        print("2. KoNLPy 추가 설정 (Ubuntu/Colab):")
        print("   !apt-get install g++ openjdk-8-jdk-headless -qq")
        print("3. 런타임 재시작 후 다시 실행")

    except Exception as e:
        print(f" 예상치 못한 오류 발생: {e}")
        print(f"오류 타입: {type(e).__name__}")

        # 구체적인 해결 방법 제시
        if "unpack" in str(e) or "NoneType" in str(e):
            print("\n 언패킹 오류 해결 방법:")
            print("1. 다음과 같이 안전하게 실행:")
            print("   result = safe_main()")
            print("   if result[0] is not None:")
            print("       classifier, dataset = result")
            print("2. 또는 빠른 테스트:")
            print("   quick_test()")

        elif "top_k" in str(e).lower():
            print("\n Top-K 메트릭 오류 해결 방법:")
            print("1. 클래스 수가 너무 적을 수 있습니다.")
            print("2. 더 많은 장르의 데이터를 추가해보세요.")
            print("3. 또는 단순한 정확도 메트릭만 사용해보세요.")

        elif "memory" in str(e).lower() or "resource" in str(e).lower():
            print("\n 메모리 부족 해결 방법:")
            print("1. 배치 크기를 줄여보세요: batch_size=2")
            print("2. max_features를 줄여보세요: max_features=1000")
            print("3. max_length를 줄여보세요: max_length=50")
            print("4. 런타임을 GPU로 변경해보세요.")

        else:
            print("\n 일반적인 문제 해결 방법:")
            print("1. 런타임 재시작: 런타임 → 런타임 재시작")
            print("2. 필수 라이브러리 재설치:")
            print("   !pip install --upgrade tensorflow keras scikit-learn")
            print("3. 빠른 테스트 실행:")
            print("   quick_test()")

        # 에러 추적 정보 출력 (디버깅용)
        import traceback
        print(f"\n 상세 오류 정보:")
        traceback.print_exc()




   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 19.4/19.4 MB 65.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 496.6/496.6 kB 13.4 MB/s eta 0:00:00
 GPU 사용 가능
 K-Pop 가사 GRU 분류기 
 K-Pop 가사 데이터셋 로드...
K-Pop 가사 데이터셋 다운로드 중...
데이터셋 다운로드 완료!
2010년부터 2023년까지의 데이터 로드 중...
[data_parser] Parsed data to DataFrame: 2010 >> 2023.
데이터 파서 방식 실패: Length mismatch: Expected axis has 0 elements, new values have 16 elements
수동 방식으로 JSON 파일들을 파싱 중...
 2017년 데이터 처리 중...
 2011년 데이터 처리 중...
 2022년 데이터 처리 중...
 2016년 데이터 처리 중...
 2012년 데이터 처리 중...
 2018년 데이터 처리 중...
 2013년 데이터 처리 중...
 2014년 데이터 처리 중...
 2015년 데이터 처리 중...
 2021년 데이터 처리 중...
 2020년 데이터 처리 중...
 2023년 데이터 처리 중...
 2019년 데이터 처리 중...
 2010년 데이터 처리 중...
 수동 로드 완료: 15744곡
 데이터 후처리 중...
중복 제거: 15744 → 4803곡
최종 데이터: 4417곡
장르별 분포:
genre
발라드           1527
댄스            1187
랩/힙합           677
발라드, 국내드라마     391
R&B/Soul       365
록/메탈           214
포크/블루스          56
Name: count, dtype: int64

 로드된 데이터 정보:
  - 총 곡 수: 4,417곡
  - 컬럼: ['title